Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Yiren Wu"
COLLABORATORS = ""

---

$$
\newcommand{\x}{\mathbf{x}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\price}{{p}}
\newcommand{\ret}{{r}}
\newcommand{\tp}{{(t)}}
\newcommand{\aapl}{{\text{AAPL}}}
\newcommand{\ba}{{\text{BA}}}
\newcommand{\spy}{{\text{SPY}}}
$$

# Assignment: Using Machine Learning for Hedging

Welcome to the first assignment !

# Problem description

We will solve a Regression task that is very common in Finance
- Given the return of "the market", predict the return of a particular stock

That is
- Given the return of a proxy for "the market" at time $t$, predict the return of, e.g., Apple at time $t$.

As we will explain,
being able to predict the relationship between two financial instruments opens up possibilities
- Use one instrument to "hedge" or reduce the risk of holding the other
- Create strategies whose returns are independent of "the market"
    - Hopefully make a profit regardless of whether the market goes up or down

## Goal

You will create models of increasing complexity in order to explain the return of Apple (ticker $\aapl$)
- The first model will have a single feature: return of the market proxy, ticker $\spy$
- Subsequent models will add the return of other tickers as additional features

## Learning Objectives
- Learn how to solve a Regression task
- Become facile in the `sklearn` toolkit for Machine Learning

## How to report your answers
We will mix explanation of the topic with tasks that you must complete. 

Look for 
the string "**Question**" to find a task that you must perform.

Most of the tasks will require you to create some code at the location indicated by

>`# YOUR CODE HERE
raise NotImplementedError()`

- Replace `raise NotImplementedError()` with your own code

# Standard imports

In [2]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn

import os
import math

%matplotlib inline

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

# Import nn_helper module
import helper
%aimport helper

helper = helper.HELPER()

# Get The data


The first step in our Recipe is Get the Data.

The data are the daily prices of a number of individual equities and equity indices.

The prices are arranged in a series in ascending date order (a timeseries).
- There are many `.csv` files for equity or index in the directory `DATA_DIR`

## API for students

We will define some utility routines to help you.

In this way, you can focus on the learning objectives rather than data manipulation.

This is not representative of the "real world"; you will need to complete data manipulation tasks in later assignments.

We provide a class `HELPER`
- Instantiated as 
>    `helper = helper.HELPER()`

With methods
- `getData`:
    - Get examples for a list of equity tickers and an index ticker.
    - Called as
    > `data = helper.getData( tickers, index_ticker, attrs)`
        - `tickers` is a list of tickers
        - `index` is the ticker of the index  
        - `attrs` is a list of data attributes
   

**Question:**
- Create code to
    - Get the adjusted close price of $\aapl$ and $\spy$ 
    - Assign the result to variable `data`

**Hint:**
- Use the `getData` method from the helper class
    - The list of tickers contains just the single ticker $\aapl$
    - The index ticker is $\spy$
    - The list of attributes is the single attribute `Adj Close`


In [2]:
helper.DATA_DIR = '../HW1_yw5418/Data_HW1/'

NameError: name 'helper' is not defined

In [5]:
# data = helper.getData( tickers, index_ticker, attrs)


ticker = "AAPL"
index_ticker = "SPY"
dateAttr = "Dt"
priceAttr = "Adj Close"

# YOUR CODE HERE
data = helper.getData(['AAPL'], "SPY", ['Adj Close'])

In [6]:
data

,AAPL_Adj_Close,SPY_Adj_Close
Dt,,
2017-01-03,110.9539,213.8428
2017-01-04,110.8297,215.1149
2017-01-05,111.3933,214.9440
2017-01-06,112.6351,215.7131
2017-01-09,113.6668,215.0010
...,...,...
2019-10-25,245.8419,301.6000
2019-10-28,248.3045,303.3000
2019-10-29,242.5618,303.2100


# Have a look at the data

We will not go through all steps in the Recipe, nor in depth.

But here's a peek at the data you retrieved

In [7]:
data.head()

,AAPL_Adj_Close,SPY_Adj_Close
Dt,,
2017-01-03,110.9539,213.8428
2017-01-04,110.8297,215.1149
2017-01-05,111.3933,214.9440
2017-01-06,112.6351,215.7131
2017-01-09,113.6668,215.0010


In [8]:
# Print the Start time and End time
print("Start time: ", data.index.min())
print("End time: ", data.index.max())

Start time:  2017-01-03
End time:  2019-10-31


Expected outputs should be similar to this:   
data:    
<table> 
    <tr> 
        <td>Dt</td><td>AAPL_Adj_Close</td><td>SPY_Adj_Close</td>
    </tr>
    <tr> 
        <td>2017-01-03</td><td>110.9539</td><td>213.8428</td> 
    </tr>
    <tr> 
        <td>2017-01-04</td><td>110.8297</td><td>215.1149</td>
    </tr>
    <tr> 
        <td>2017-01-05</td><td>111.3933</td><td>214.9440</td>
    </tr>
    <tr> 
        <td>2017-01-06</td><td>112.6351</td><td>215.7131</td> 
    </tr>
    <tr> 
        <td>2017-01-09</td><td>113.6668</td> <td>215.0010</td>
    </tr>
</table>   

## Create DataFrame of price levels for the training examples

The training examples will be stored in a DataFrame.

- The DataFrame should have two columns: the price level for the ticker and for the index
- The minimum date in the DataFrame should be **the trading day before** `start_dt`
    - That is: the latest date for which there is data and which is less than `start_dt`
    - For example, if `start_dt` is a Monday, the "day before" would be Friday, not Sunday.
        - Similarly for the case where the day before `start_dt` is a holiday
- The maximum date in the DataFrame should be `end_dt`

The reason we are adding one day prior to `start_dt`
- We want to have returns (percent price changes) from `start_dt` onwards
- In order to compute a return for `start_dt`, we need the level from the prior day

**Question:**

- Complete the function `getRange()`
    - To return the subset of rows of our examples
    - Beginning on the **trading day before** date `start_dt`
    - Ending on date `end_dt`


In [9]:
start_dt = "2018-01-02"
end_dt = "2018-09-28"
train_data_price = None

# Set variable train_data_price to be a DataFrame with two columns
## AAPL_Adj_Close, SPY_Adj_Close
## with dates as the index
## Having minimum date equal to THE DAY BEFORE start_dt
## Having maximum date equal to end_dt

def getRange(df, start_dt, end_dt):
    '''
    Return the the subset of rows of DataFrame df
    restricted to dates between start_dt and end_dt
    
    Parameters
    ----------
    start_dt: String
    - Start date
    
    end_dt: String
    - End date
    '''
    # YOUR CODE HERE
    return (df.iloc[len(df.loc[:start_dt])-2:]).loc[:end_dt]
    

train_data_price = getRange(data, start_dt, end_dt)
print(train_data_price.head())

            AAPL_Adj_Close  SPY_Adj_Close
Dt                                       
2017-12-29        164.2589       258.2823
2018-01-02        167.1999       260.1310
2018-01-03        167.1708       261.7763
2018-01-04        167.9473       262.8796
2018-01-05        169.8594       264.6314


As you can see, each row has two attributes for one  date
- Price (adjusted close) of ticker $\aapl$
- Price (adjusted close) of the market proxy $\spy$

# Create test set

We just created a set of training examples as a subset of the rows of `data`.

We will do the same to create a set of test examples.

**Question:**

Set variable `test_data_price`
- To  the subset of rows of our examples
- Beginning on the **trading day before** date `test_start_dt`
- Ending on date `test_end_dt`

**Hint**
- Use `getRange` with different arguments for the dates

In [10]:
test_start_dt = '2018-10-01'
test_end_dt = '2018-12-31'

# YOUR CODE HERE

test_data_price = getRange(data, test_start_dt, test_end_dt)

In [11]:
test_data_price

,AAPL_Adj_Close,SPY_Adj_Close
Dt,,
2018-09-28,221.6252,285.0555
2018-10-01,223.1175,286.0458
2018-10-02,225.1006,285.8791
2018-10-03,227.8398,286.0359
2018-10-04,223.8342,283.8004
...,...,...
2018-12-24,144.6565,231.1158
2018-12-26,154.8435,242.7929
2018-12-27,153.8386,244.6569


# Prepare the data

In Finance, it is very typical to work with *relative changes* (e.g., percent price change)
rather than *absolute changes* (price change) or *levels* (prices).

Without going into too much detail
- Relative changes are more consistent over time than either absolute changes or levels
- The consistency can facilitate the use of data over a longer time period

For example, let's suppose that prices are given in units of USD (dollar)
- A price change of 1 USD is more likely for a stock with price level 100 than price level 10
    - A relative change of $1/100 = 1 %$ is more likely than a change of $1/10 = 10%$
    - So relative changes are less dependent on price level than either price changes or price levels
    
    
To compute the *return* (percent change in prices)
 for ticker $\aapl$ (Apple) on date $t$

$$
\begin{array}[lll]\\
\ret_\aapl^\tp = \frac{\price _\aapl^\tp}{\price _\aapl^{(t-1)}} -1 \\
\text{where} \\
\price_\aapl^\tp \text{ denotes the price of ticker } \aapl \text{ on date } t \\
\ret_\aapl^\tp \text{ denotes the return of ticker } \aapl \text{ on date } t
\end{array}
$$


# Transformations: transform the training data

Our first task is to transform the data from price levels (Adj Close)
to Percent Price Changes.

Moreover, the date range for the training data is specified to be in the range
from `start_dt` (start date) to `end_dt`, inclusive on both sides.

**Note**

We will need to apply **identical** transformations to both the training and test data examples.

In the cells that immediately follow, we will do this only for the **training data**

You will need to repeat these steps for the test data in a subsequent step.

You are well-advised to create subroutines or functions to accomplish these tasks !
- You will apply them first to transform training data
- You will apply them a second time to transform the test data

We will achieve this is several steps

## Create DataFrame of returns for training examples

Create a new DataFrame with percent price changes of the columns, rather than the levels

**Question:**
- Complete function `getReturns()` to compute the returns of tickers.
- Set variable `train_data_ret` to be a DataFrame with the same columns
    - But where the prices have been replaced by day over day percent changes
    - The column names of `train_data_ret` should be the same as the original columns names
    - We give you code to rename the columns to reflect the changed meaning of the data in the next step

**Hint:**
- look up the Pandas `pct_change()` method    

In [12]:
train_data_df = None

def getReturns(df, start_dt, end_dt):
    '''
    Return the day over day percent changes of adjusted price
    '''
    # YOUR CODE HERE
    return ((df.iloc[len(df.loc[:start_dt])-2:]).loc[:end_dt]).pct_change().dropna()

train_data_ret = getReturns(train_data_price, start_dt, end_dt)
print(train_data_ret.head())

            AAPL_Adj_Close  SPY_Adj_Close
Dt                                       
2018-01-02        0.017905       0.007158
2018-01-03       -0.000174       0.006325
2018-01-04        0.004645       0.004215
2018-01-05        0.011385       0.006664
2018-01-08       -0.003714       0.001829


In [13]:
train_data_ret

,AAPL_Adj_Close,SPY_Adj_Close
Dt,,
2018-01-02,0.017905,0.007158
2018-01-03,-0.000174,0.006325
2018-01-04,0.004645,0.004215
2018-01-05,0.011385,0.006664
2018-01-08,-0.003714,0.001829
...,...,...
2018-09-24,0.014380,-0.003322
2018-09-25,0.006341,-0.000928
2018-09-26,-0.007966,-0.002992


Since the columns of `train_data_ret` are now returns, we will rename then for you.

Also, we will drop the earliest date
- There is now return for this date
- We included this row only so we could compute the return for the following trading date

In [14]:
## Rename the columns to indicate that they have been transformed from price (Adj_close) to Return
train_data_ret = helper.renamePriceToRet( train_data_ret )

## Drop the first date (the day before `start_dt`) since it has an undefined return
train_data_ret = train_data_ret[ start_dt:]
print(train_data_ret.head())

            AAPL_Ret   SPY_Ret
Dt                            
2018-01-02  0.017905  0.007158
2018-01-03 -0.000174  0.006325
2018-01-04  0.004645  0.004215
2018-01-05  0.011385  0.006664
2018-01-08 -0.003714  0.001829


Expected outputs should be similar to this:   


## Remove the target 

The only feature is the return of the market proxy $\spy$.

Predicting the target given the target as a feature would be cheating !

So we will create `X_train, y_train` from `train_data_ret`
- `X_train` has only features for the example
- `y_train` is the target for the example

In [15]:
indexAttr = index_ticker + "_Ret"

X_train, y_train =  train_data_ret[ [indexAttr] ], train_data_ret.drop(columns=[ indexAttr ] )

In [16]:
X_train
y_train

,SPY_Ret
Dt,
2018-01-02,0.007158
2018-01-03,0.006325
2018-01-04,0.004215
2018-01-05,0.006664
2018-01-08,0.001829
...,...
2018-09-24,-0.003322
2018-09-25,-0.000928
2018-09-26,-0.002992


,AAPL_Ret
Dt,
2018-01-02,0.017905
2018-01-03,-0.000174
2018-01-04,0.004645
2018-01-05,0.011385
2018-01-08,-0.003714
...,...
2018-09-24,0.014380
2018-09-25,0.006341
2018-09-26,-0.007966


# Transformations: transform the test data

We have just performed some transformations of the training data.

**Remember**:

You need to perform *identical* transformations to the test data.

The test data will be returns from `test_start_dt` to `test_end_dt` inclusive.

We will apply identical transformations as we did to the training data, but with a different date range.


We obtained `X_train, y_train` via transformations to `train_data_price`.

We will now obtain `X_test, y_test` by identical transformations to `test_data_price`

**Question:**

Create the training data `X_test, y_test`
- Apply the same transformations to `test_data_price` as you did to `train_data_price`
- To create variable `test_data_ret`
- We will convert `test_data_ret` to `X_test, y_test` for you

**Hints**

Create `test_data_ret` in a manner analogous to the creation of `train_data_ret`
- Use `getReturns` to convert price levels to returns
- Use `renamePriceToRet` to rename the columns to reflect the change in data from price to return
- Drop the first date from `test_data_ret` as it has an undefined return


In [17]:
test_data_ret = None
X_test = None
y_test = None

# YOUR CODE HERE
test_data_ret = getReturns(data, test_start_dt, test_end_dt)

test_data_ret = helper.renamePriceToRet(test_data_ret)


X_test, y_test =  test_data_ret[ [indexAttr] ], test_data_ret.drop(columns=[ indexAttr ] )

print("test data length", test_data_ret.shape[0])
print("X test length", X_test.shape[0])
print("y test length", y_test.shape[0])
print(test_data_ret.head())

test data length 63
X test length 63
y test length 63
            AAPL_Ret   SPY_Ret
Dt                            
2018-10-01  0.006733  0.003474
2018-10-02  0.008888 -0.000583
2018-10-03  0.012169  0.000548
2018-10-04 -0.017581 -0.007815
2018-10-05 -0.016229 -0.005597


Expected outputs should be similar to this:   

test data length 63      
X test length 63      
y test length 63       
data:    
<table> 
    <tr> 
        <td>Dt</td><td>AAPL_Ret</td><td>SPY_Ret</td>
    </tr>
    <tr> 
        <td>2018-10-01</td><td>0.006733</td><td>0.003474</td>
    </tr>
    <tr> 
        <td>2018-10-02</td><td>0.008888</td><td>-0.000583</td>
    </tr>
    <tr> 
        <td>2018-10-03</td><td>0.012169</td><td>0.000548</td> 
    </tr>
    <tr> 
        <td>2018-10-04</td><td>-0.017581</td><td>-0.007815</td>
    </tr>
    <tr> 
        <td>2018-10-05</td><td>-0.016229</td><td>-0.005597</td>
    </tr>
</table>   

# Train a model (Regression)

Use Linear Regression to predict the return of a ticker from the return of the market proxy $\spy$.
For example, for ticker $\aapl$

$$
\ret_\aapl^\tp = \beta_0 + \beta_{\aapl, \spy} * \ret_\spy^\tp + \epsilon_{\aapl}^\tp
$$

Each example corresponds to one day (time $t$)
- has features
    - constant 1, corresponding to the intercept parameter
    - return of the market proxy $\spy$
       $$\x^\tp = \begin{pmatrix}
        1 \\
        \ret_\spy^\tp
        \end{pmatrix}$$

- has target
    - return of the ticker
    $$\y^\tp = \ret_\aapl^\tp$$

 
You will use Linear Regression to solve for parameters $\beta_0$,  $\beta_{\aapl, \spy}$ 

- In the lectures we used the symbol $\Theta$ to denote the parameter vector; here we use $\mathbf{\beta}$
- In Finance the symbol $\beta$ is often used to denote the relationship between returns.
- Rather than explicitly creating a constant 1 feature
    - you may invoke the model object with the option including an intercept
    - if you do so, the feature vector you pass will be
   $$\x^\tp = \begin{pmatrix}
        \ret_\spy^\tp
        \end{pmatrix}$$  
    


- Use the entire training set
- Do not use cross-validation

**Questions:**
- Complete the function `createModel()` to build your linear regression model
- Complete the function `regress()` to do regression and return intercept and coefficients
- Replace the 0 values in the following cell with your answers, and execute the print statements


In [18]:
from sklearn import datasets, linear_model

beta_0 = 0    # The regression parameter for the constant
beta_SPY = 0  # The regression parameter for the return of SPY
ticker = "AAPL"

def createModel():
    '''
    Build your linear regression model using sklearn
    
    Returns
    -------
    An sklearn model object implementing Linear Regression
    '''
    # YOUR CODE HERE
    return linear_model.LinearRegression(fit_intercept = True)

def regress(model, X, y):
    '''
    Do regression using returns of your ticker and index
    
    Parameters
    -----------
    model: model object implementing Linear Regression
    X: ticker returns
    y: index returns
    
    Returns
    -------
    Tuple (intercept, slope)
    where
    - interecept is the value of the intercept term
    - slope is the value of the slope
    '''
    # YOUR CODE HERE
    model.fit(X,y)
    return model.intercept_[0], model.coef_[0][0]
    

# Assign to answer variables
regr = createModel()

beta_0, beta_SPY = regress(regr, X_train, y_train)


print("{t:s}: beta_0={b0:3.3f}, beta_SPY={b1:3.3f}".format(t=ticker, b0=beta_0, b1=beta_SPY))

AAPL: beta_0=0.001, beta_SPY=1.071


Your expected outputs should be:
<table> 
    <tr> 
        <td>  
            beta_0
        </td>
        <td>
         0.001
        </td>
    </tr>
    <tr> 
        <td>
            beta_SPY
        </td>
        <td>
         1.071
        </td>
    </tr>

</table>

## Train the model using Cross validation

Use 5-fold cross validation

**Question:**
- Complete the function `compute_cross_val_avg()` to compute the average score of 5-fold cross validation
- Replace the 0 values in the following cell with your answers, and execute the print statements

**Hint:**  
- You can use the `cross_val_score` in `sklearn.model_selection`

In [19]:
from sklearn.model_selection import cross_val_score

cross_val_avg = 0 # average score of cross validation
k = 5             # 5-fold cross validation

def compute_cross_val_avg(model, X, y, k):
    '''
    Compute the average score of k-fold cross validation
    
    Parameters
    -----------
    model: model you build with method "createModel()"
    X: ticker returns
    y: index returns
    k: k-fold cross validation
    
    Returns
    --------
    The average, across the k iterations, of the score
    '''
    # YOUR CODE HERE
    return np.average(cross_val_score(model, X, y, cv = k))

    
cross_val_avg = compute_cross_val_avg(regr, X_train, y_train, 5)
print("{t:s}: Avg cross val score = {sc:3.2f}".format(t=ticker, sc=cross_val_avg) )

AAPL: Avg cross val score = 0.33


## Evaluate Loss (in sample RMSE) and Performance (Out of sample RMSE)

**Question:**
- Complete the function `computeRMSE()` to calculate the Root of Mean Square Error (RMSE)
- Replace the 0 values in the following cell with your answers, and execute the print statements

In [20]:
from sklearn.metrics import mean_squared_error

rmse_in_sample = 0 # in sample loss
rmse_out_sample = 0 # out of sample performance

# Predicted  in-sample returns of AAPL using SPY index
aapl_predicted_in_sample = regr.predict(X_train)
# Predicted out-of-sample returns of AAPL using SPY index
aapl_predicted_out_sample = regr.predict(X_test)

def computeRMSE( target, predicted ):
    '''
    Calculate the RMSE
    
    Parameters
    ----------
    target: actual ticker returns
    predicted: predicted ticker returns
    
    Returns
    --------
    The value of the RMSE
    '''
    # YOUR CODE HERE
    return mean_squared_error(target, predicted, squared = False)
    
    
rmse_in_sample = computeRMSE(y_train, aapl_predicted_in_sample)
rmse_out_sample = computeRMSE(y_test, aapl_predicted_out_sample)

print("In Sample Root Mean squared error: {:.3f}".format( rmse_in_sample ) )
print("Out of Sample Root Mean squared error: {:.3f}".format( rmse_out_sample ) )

In Sample Root Mean squared error: 0.011
Out of Sample Root Mean squared error: 0.015


## Hedged returns

Why is being able to predict the return of a ticker, given the return of another instrument (e.g., the market proxy) useful ?
- It **does not** allow us to predict the future
    - To predict $\ret_\aapl^\tp$, we require the same day return of the proxy $\ret_\spy$
- It **does** allow us to predict how much $\aapl$ will outperform the market proxy

Consider an investment that goes long (i.e, holds a positive quantity of $\aapl$
- Since the relationship between returns is positive
    - You will likely make money if the market goes up
    - You will likely lose money if the market goes down
    
Consider instead a *hedged* investment
- Go long 1 USD of $\aapl$
- Go short (hold a negative quantity) $\beta_{\aapl,\spy}$ USD of the market proxy $\spy$

Your *hedged return* on this long/short portfolio will be
$$
{\ret'}_{\aapl}^\tp = \ret_\aapl^\tp - \beta_{\aapl, \spy} * \ret_\spy^\tp
$$

As long as
$$
\ret_\aapl^\tp \gt \beta_{\aapl, \spy} * \ret_\spy^\tp
$$
you will make a profit, regardless of whether the market proxy rises or falls !

That is: you make money as long as $\aapl$ *outperforms* the market proxy.


This hedged portfolio is interesting
- Because your returns are independent of the market
- The volatility of your returns is likely much lower than the volatility of the long-only investment
- There is a belief that it is difficult to predict the market $\ret_\spy$
- But you might be able to discover a ticker (e.g., $\aapl$) that will outpeform the market

This is a real world application of the Regression task in Finance.

## Compute the hedged return on the test data examples
$$
{\ret'}_{\aapl}^\tp = \ret_\aapl^\tp - \beta_{\aapl, \spy} * \ret_\spy^\tp
$$
for all dates $t$ in the **test set**.  

**Question:**
- Complete the function `compute_hedged_series` 
- It should use the model results and the **test examples** to return
- The hedged returns of the test examples

**Hints**
- An `sklearn` model, once fitted, may have attributes `coef_` that give you access to the parameters

In [21]:
hedged_series = pd.DataFrame()

def compute_hedged_series(model, X, y):
    '''
    Compute the hedged series
    
    Parameters
    ----------
    model: model you build with method "createModel()"
    X: index returns 
    y: ticker returns 
    
    Returns
    --------
    Timeseries of hedged returns (i.e., return of portfolio that is long "ticker" and short index)
    '''
    # YOUR CODE HERE
    return y['AAPL_Ret'] - model.coef_[0][0] * X['SPY_Ret']
    

hedged_series = compute_hedged_series(regr, X_test, y_test)
print(hedged_series.head())

Dt
2018-10-01    0.003012
2018-10-02    0.009512
2018-10-03    0.011581
2018-10-04   -0.009208
2018-10-05   -0.010233
dtype: float64


# A model with more than one feature

Our simple model used a single feature (return of "the market") to make predictions.
- There are many more timeseries, stored as .CSV files, in the data directory

**Question**
- Construct a model with *more than one* feature by choosing from among these timeseries
- *Explain* why/how you chose the additional features
    - You may run multiple experiments if you choose
    - **Remember:** your notebook is a *movie*; we want to see your journey to your solution, not just the last step
- Report the average of the scores when using 5 fold cross-validation
- Report the in-sample and out of sample RMSE



### My first step is to choose uncorrelated features, so I will calculate the correlation martix first

In [22]:
data = helper.getData(['AAPL', 'ACN', 'ADBE', 'CRM', 'CSCO','IBM', 'INTC', 'MA','MSFT'
                      , 'NVDA', 'V', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE',
                      'XLU', 'XLV', 'XLY'], "SPY", ['Adj Close'])

In [23]:
data

,AAPL_Adj_Close,ACN_Adj_Close,ADBE_Adj_Close,CRM_Adj_Close,CSCO_Adj_Close,IBM_Adj_Close,INTC_Adj_Close,MA_Adj_Close,MSFT_Adj_Close,NVDA_Adj_Close,...,XLE_Adj_Close,XLF_Adj_Close,XLI_Adj_Close,XLK_Adj_Close,XLP_Adj_Close,XLRE_Adj_Close,XLU_Adj_Close,XLV_Adj_Close,XLY_Adj_Close,SPY_Adj_Close
Dt,,,,,,,,,,,,,,,,,,,,,
2017-01-03,110.9539,110.7381,103.48,70.54,27.8299,146.9351,33.8738,103.4719,59.4966,101.0116,...,69.7149,22.3628,59.3227,46.9054,48.1496,28.2050,44.2966,66.9012,79.0179,213.8428
2017-01-04,110.8297,111.0044,104.14,72.80,27.6644,148.7543,33.6979,104.4537,59.2304,103.3683,...,69.5685,22.5435,59.6733,47.0689,48.1496,28.5616,44.4612,67.4280,80.0698,215.1149
2017-01-05,111.3933,109.3403,105.91,72.79,27.7288,148.2621,33.6424,105.2604,59.2304,100.7443,...,69.3946,22.3152,59.5027,47.1458,48.3073,28.7079,44.5069,67.7729,80.0119,214.9440
2017-01-06,112.6351,110.5860,108.30,73.80,27.7839,148.9916,33.7627,106.0180,59.7438,102.0910,...,69.4586,22.3913,59.8439,47.4919,48.3537,28.7079,44.6440,67.9645,80.4075,215.7131
2017-01-09,113.6668,109.3499,108.57,73.96,27.7379,147.3393,33.8830,105.8114,59.5536,106.2301,...,68.4518,22.2391,59.3795,47.4823,47.9640,28.5250,44.0589,68.2519,80.3400,215.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-25,245.8419,183.0700,270.98,150.49,46.9000,133.8590,56.1510,270.1900,140.7300,204.5400,...,59.5200,28.6400,78.6000,82.9200,61.0900,39.4900,64.1600,92.0600,121.0600,301.6000
2019-10-28,248.3045,182.8700,271.35,154.79,47.1700,134.3828,56.4493,275.9300,144.1900,206.7900,...,59.1800,28.7900,78.8200,83.9700,60.9700,39.0700,63.2800,93.0100,121.4400,303.3000
2019-10-29,242.5618,184.8000,271.45,155.09,47.2400,132.2579,56.0316,274.2900,142.8300,202.9200,...,59.4200,28.8600,79.0800,83.2100,60.9700,39.1700,63.3400,94.3300,120.7500,303.2100


In [24]:
# calculate the correlation matrix
corr = helper.getData(['ACN', 'ADBE', 'CRM', 'CSCO','IBM', 'INTC', 'MA','MSFT'
                      , 'NVDA', 'V', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE',
                      'XLU', 'XLV', 'XLY'], "SPY", ['Adj Close']).pct_change().dropna().corr()

In [25]:
corr

,ACN_Adj_Close,ADBE_Adj_Close,CRM_Adj_Close,CSCO_Adj_Close,IBM_Adj_Close,INTC_Adj_Close,MA_Adj_Close,MSFT_Adj_Close,NVDA_Adj_Close,V_Adj_Close,...,XLE_Adj_Close,XLF_Adj_Close,XLI_Adj_Close,XLK_Adj_Close,XLP_Adj_Close,XLRE_Adj_Close,XLU_Adj_Close,XLV_Adj_Close,XLY_Adj_Close,SPY_Adj_Close
ACN_Adj_Close,1.000000,0.575199,0.547874,0.588514,0.476528,0.467711,0.687772,0.668044,0.424822,0.678227,...,0.458899,0.593854,0.661927,0.722507,0.500175,0.378493,0.196971,0.636446,0.668012,0.747758
ADBE_Adj_Close,0.575199,1.000000,0.762324,0.551057,0.415243,0.437711,0.723717,0.764803,0.524247,0.735260,...,0.359178,0.432416,0.557841,0.773755,0.322341,0.305882,0.133670,0.585521,0.647265,0.691750
CRM_Adj_Close,0.547874,0.762324,1.000000,0.552158,0.385647,0.424306,0.725366,0.716346,0.520539,0.717449,...,0.341984,0.455720,0.541594,0.762494,0.300230,0.265962,0.104154,0.581091,0.659446,0.684079
CSCO_Adj_Close,0.588514,0.551057,0.552158,1.000000,0.491489,0.532606,0.605468,0.649091,0.460491,0.595314,...,0.490555,0.576930,0.653779,0.735646,0.443626,0.313736,0.159958,0.625808,0.684007,0.747993
IBM_Adj_Close,0.476528,0.415243,0.385647,0.491489,1.000000,0.391093,0.450129,0.481747,0.340424,0.457485,...,0.406845,0.513688,0.551577,0.559915,0.406958,0.284587,0.154300,0.496708,0.543889,0.601749
INTC_Adj_Close,0.467711,0.437711,0.424306,0.532606,0.391093,1.000000,0.501648,0.576634,0.506291,0.504932,...,0.417161,0.478565,0.553715,0.683589,0.320901,0.235932,0.105963,0.493422,0.569187,0.635341
MA_Adj_Close,0.687772,0.723717,0.725366,0.605468,0.450129,0.501648,1.000000,0.768390,0.548843,0.885365,...,0.432334,0.546884,0.618585,0.844355,0.377682,0.344629,0.129114,0.606791,0.718768,0.771851
MSFT_Adj_Close,0.668044,0.764803,0.716346,0.649091,0.481747,0.576634,0.768390,1.000000,0.541659,0.766305,...,0.457936,0.575529,0.646512,0.883731,0.419431,0.367478,0.167054,0.655716,0.757177,0.813725
NVDA_Adj_Close,0.424822,0.524247,0.520539,0.460491,0.340424,0.506291,0.548843,0.541659,1.000000,0.540084,...,0.366355,0.448045,0.526198,0.685618,0.236205,0.173400,-0.023261,0.446260,0.567996,0.604066
V_Adj_Close,0.678227,0.735260,0.717449,0.595314,0.457485,0.504932,0.885365,0.766305,0.540084,1.000000,...,0.451409,0.573432,0.637690,0.837222,0.409613,0.360624,0.132309,0.620122,0.715317,0.781659


In [26]:
corr[corr < 0.5]

,ACN_Adj_Close,ADBE_Adj_Close,CRM_Adj_Close,CSCO_Adj_Close,IBM_Adj_Close,INTC_Adj_Close,MA_Adj_Close,MSFT_Adj_Close,NVDA_Adj_Close,V_Adj_Close,...,XLE_Adj_Close,XLF_Adj_Close,XLI_Adj_Close,XLK_Adj_Close,XLP_Adj_Close,XLRE_Adj_Close,XLU_Adj_Close,XLV_Adj_Close,XLY_Adj_Close,SPY_Adj_Close
ACN_Adj_Close,NaN,NaN,NaN,NaN,0.476528,0.467711,NaN,NaN,0.424822,NaN,...,0.458899,NaN,NaN,NaN,NaN,0.378493,0.196971,NaN,NaN,NaN
ADBE_Adj_Close,NaN,NaN,NaN,NaN,0.415243,0.437711,NaN,NaN,NaN,NaN,...,0.359178,0.432416,NaN,NaN,0.322341,0.305882,0.133670,NaN,NaN,NaN
CRM_Adj_Close,NaN,NaN,NaN,NaN,0.385647,0.424306,NaN,NaN,NaN,NaN,...,0.341984,0.455720,NaN,NaN,0.300230,0.265962,0.104154,NaN,NaN,NaN
CSCO_Adj_Close,NaN,NaN,NaN,NaN,0.491489,NaN,NaN,NaN,0.460491,NaN,...,0.490555,NaN,NaN,NaN,0.443626,0.313736,0.159958,NaN,NaN,NaN
IBM_Adj_Close,0.476528,0.415243,0.385647,0.491489,NaN,0.391093,0.450129,0.481747,0.340424,0.457485,...,0.406845,NaN,NaN,NaN,0.406958,0.284587,0.154300,0.496708,NaN,NaN
INTC_Adj_Close,0.467711,0.437711,0.424306,NaN,0.391093,NaN,NaN,NaN,NaN,NaN,...,0.417161,0.478565,NaN,NaN,0.320901,0.235932,0.105963,0.493422,NaN,NaN
MA_Adj_Close,NaN,NaN,NaN,NaN,0.450129,NaN,NaN,NaN,NaN,NaN,...,0.432334,NaN,NaN,NaN,0.377682,0.344629,0.129114,NaN,NaN,NaN
MSFT_Adj_Close,NaN,NaN,NaN,NaN,0.481747,NaN,NaN,NaN,NaN,NaN,...,0.457936,NaN,NaN,NaN,0.419431,0.367478,0.167054,NaN,NaN,NaN
NVDA_Adj_Close,0.424822,NaN,NaN,0.460491,0.340424,NaN,NaN,NaN,NaN,NaN,...,0.366355,0.448045,NaN,NaN,0.236205,0.173400,-0.023261,0.446260,NaN,NaN
V_Adj_Close,NaN,NaN,NaN,NaN,0.457485,NaN,NaN,NaN,NaN,NaN,...,0.451409,NaN,NaN,NaN,0.409613,0.360624,0.132309,NaN,NaN,NaN


### By looking at the above value, we will choose the ticker whose value is uncorrelated to most of the other features. We will select IBM, INTC, XLRE, XLU, XLP as our features.

In [27]:
data =  helper.getData(['IBM', "INTC", 'XLU', 'XLP'], "AAPL", ['Adj Close'])

In [28]:
data

,IBM_Adj_Close,INTC_Adj_Close,XLU_Adj_Close,XLP_Adj_Close,AAPL_Adj_Close
Dt,,,,,
2017-01-03,146.9351,33.8738,44.2966,48.1496,110.9539
2017-01-04,148.7543,33.6979,44.4612,48.1496,110.8297
2017-01-05,148.2621,33.6424,44.5069,48.3073,111.3933
2017-01-06,148.9916,33.7627,44.6440,48.3537,112.6351
2017-01-09,147.3393,33.8830,44.0589,47.9640,113.6668
...,...,...,...,...,...
2019-10-25,133.8590,56.1510,64.1600,61.0900,245.8419
2019-10-28,134.3828,56.4493,63.2800,60.9700,248.3045
2019-10-29,132.2579,56.0316,63.3400,60.9700,242.5618


### We split our dataset into training and testing set

In [29]:
train_set = getReturns(data, start_dt, end_dt)
test_set = getReturns(data, test_start_dt, test_end_dt)
train_data_ret = helper.renamePriceToRet( train_set )
test_data_ret = helper.renamePriceToRet( test_set )

In [30]:
train_data_ret

,IBM_Ret,INTC_Ret,XLU_Ret,XLP_Ret,AAPL_Ret
Dt,,,,,
2018-01-02,0.005410,0.014946,-0.009301,-0.006153,0.017905
2018-01-03,0.027488,-0.033938,-0.007855,-0.000355,-0.000174
2018-01-04,0.020253,-0.018338,-0.008306,0.002832,0.004645
2018-01-05,0.004886,0.006977,-0.000388,0.004411,0.011385
2018-01-08,0.006031,0.000000,0.009350,0.002459,-0.003714
...,...,...,...,...,...
2018-09-24,-0.008722,0.005356,-0.008859,-0.015268,0.014380
2018-09-25,-0.007464,-0.021316,-0.013689,-0.005905,0.006341
2018-09-26,0.018132,-0.004574,-0.009833,0.001113,-0.007966


### In the training and testing, we will define features and the target

In [31]:
y_train_set = train_data_ret['AAPL_Ret']
X_train_set = train_data_ret.iloc[:,:4]
y_test_set = test_data_ret['AAPL_Ret']
X_test_set = test_data_ret.iloc[:,:4]


### create the model

In [32]:
regr = createModel()
regr.fit(X_train_set,y_train_set)
regr.intercept_
regr.coef_


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

0.0015968489764211944

array([ 0.2438646 ,  0.23592007, -0.05851487,  0.23446988])

### Compute the average of the scores when using 5 fold cross-validation

In [33]:
corss_val_avg = compute_cross_val_avg(regr, X_train_set, y_train_set, 5)
corss_val_avg

0.12375233046140766

### Report the in-sample and out of sample RMSE 

In [34]:
aapl_predicted_in_sample = regr.predict(X_train_set)
aapl_predicted_out_sample = regr.predict(X_test_set)


rmse_in_sample = computeRMSE(y_train, aapl_predicted_in_sample)
rmse_out_sample = computeRMSE(y_test, aapl_predicted_out_sample)

print("In Sample Root Mean squared error: {:.3f}".format( rmse_in_sample ) )
print("Out of Sample Root Mean squared error: {:.3f}".format( rmse_out_sample ) )

In Sample Root Mean squared error: 0.012
Out of Sample Root Mean squared error: 0.020
